In [2]:
import numpy as np
import pandas as pd
import math
import pprint
import time


# load and store dataset from website
data = pd.io.parsers.read_csv(filepath_or_buffer='iris.data.txt',header=None, sep=',',
    )

print(data)
print(type(data[0][0]))



       0    1    2    3               4
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9  3.0  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4    5.0  3.6  1.4  0.2     Iris-setosa
..   ...  ...  ...  ...             ...
145  6.7  3.0  5.2  2.3  Iris-virginica
146  6.3  2.5  5.0  1.9  Iris-virginica
147  6.5  3.0  5.2  2.0  Iris-virginica
148  6.2  3.4  5.4  2.3  Iris-virginica
149  5.9  3.0  5.1  1.8  Iris-virginica

[150 rows x 5 columns]
<class 'numpy.float64'>


In [29]:
# split up 80% of dataset for training and 20% for testing
train = data[0:40].append(data[50:90]).append(data[100:140])
test  = data[40:50].append(data[90:100]).append(data[140:150])
print(train.shape)
print(test.shape)

covariance = np.cov(train)

# categorize our flowers by their respective class and create separate tables for them
setosa = train[train[4] == 'Iris-setosa']
print(setosa.tail(),'\n')
versicolor = train[train[4] == 'Iris-versicolor']
print(versicolor.tail(),'\n')
virginica = train[train[4] == 'Iris-virginica']
print(virginica.tail())

      0    1    2    3            4
35  5.0  3.2  1.2  0.2  Iris-setosa
36  5.5  3.5  1.3  0.2  Iris-setosa
37  4.9  3.1  1.5  0.1  Iris-setosa
38  4.4  3.0  1.3  0.2  Iris-setosa
39  5.1  3.4  1.5  0.2  Iris-setosa 

      0    1    2    3                4
85  6.0  3.4  4.5  1.6  Iris-versicolor
86  6.7  3.1  4.7  1.5  Iris-versicolor
87  6.3  2.3  4.4  1.3  Iris-versicolor
88  5.6  3.0  4.1  1.3  Iris-versicolor
89  5.5  2.5  4.0  1.3  Iris-versicolor 

       0    1    2    3               4
135  7.7  3.0  6.1  2.3  Iris-virginica
136  6.3  3.4  5.6  2.4  Iris-virginica
137  6.4  3.1  5.5  1.8  Iris-virginica
138  6.0  3.0  4.8  1.8  Iris-virginica
139  6.9  3.1  5.4  2.1  Iris-virginica


C:\Users\timlu\AppData\Local\Temp\ipykernel_22808\1682074540.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = data[0:40].append(data[50:90]).append(data[100:140])
C:\Users\timlu\AppData\Local\Temp\ipykernel_22808\1682074540.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test  = data[40:50].append(data[90:100]).append(data[140:150])


In [3]:

# drop labels so that we only have numerical data for training sets
train_setosa = setosa.drop(4,axis=1)
print('setosa:')
print(train_setosa.head(),'\n')
train_versicolor = versicolor.drop(4,axis=1)
print('versicolor:')
print(train_versicolor.head(),'\n')
train_virginica = virginica.drop(4,axis=1)
print('virginica:')
print(train_virginica.head(),'\n')






NameError: name 'setosa' is not defined

In [38]:
# define the probability density function (as noted from Dr. Fang's lectures)
def P(x,mean,covariance):
    k = x.shape[0]
    first = 1/math.sqrt(((2.0*math.pi)**k) * np.linalg.det(covariance)) 
    second = math.exp(-0.5*np.dot(np.dot((x-mean),np.linalg.inv(covariance)),(x-mean)[np.newaxis].T))
    return first*second
# in order to use the function from above, we have to calculate the mean for each of the flowers' attributes
def my_means(matrix):
    means = []
    for attribute in matrix.values.T:
        means.append(attribute.sum()/float(matrix.shape[0]))
    return np.array(means)



In [39]:
# we also need to calculate the covariances
def my_covs(matrix):
    mean = my_means(matrix)
    # setup each class to have a 4 x 4 matrix for covariance
    total_cov = np.zeros((matrix.shape[1],matrix.shape[1]))
    for row in range(matrix.shape[0]):
        total_cov += np.outer((matrix.iloc[row].values - mean),(matrix.iloc[row].values - mean))
    cov = total_cov/float(matrix.shape[0])
    return cov

In [40]:
means = {}
means['setosa']  = my_means(train_setosa)
means['versicolor'] = my_means(train_versicolor)
means['virginica'] = my_means(train_virginica)

covs = {}
covs['setosa'] = my_covs(train_setosa)
covs['versicolor'] = my_covs(train_versicolor)
covs['virginica'] = my_covs(train_virginica)

# this variable is necessary for LDA, since the covariances are assumed to be equal
cov_avg = (covs['setosa'] + covs['versicolor'] + covs['virginica'])/3.0

# pprint is a module that prints out dictionaries all pretty :D
print("Means:")
pprint.pprint(means)
print("\nCovariances:")
pprint.pprint(covs)

Means:
{'setosa': array([5.0375, 3.44  , 1.4625, 0.2325]),
 'versicolor': array([6.01  , 2.78  , 4.3175, 1.35  ]),
 'virginica': array([6.6225, 2.96  , 5.6075, 1.99  ])}

Covariances:
{'setosa': array([[0.12784375, 0.0965    , 0.01265625, 0.01328125],
       [0.0965    , 0.1294    , 0.002     , 0.0142    ],
       [0.01265625, 0.002     , 0.02884375, 0.00446875],
       [0.01328125, 0.0142    , 0.00446875, 0.00969375]]),
 'versicolor': array([[0.2669    , 0.08445   , 0.167825  , 0.051     ],
       [0.08445   , 0.1081    , 0.07885   , 0.04425   ],
       [0.167825  , 0.07885   , 0.19844375, 0.071875  ],
       [0.051     , 0.04425   , 0.071875  , 0.042     ]]),
 'virginica': array([[0.45624375, 0.10765   , 0.34883125, 0.049975  ],
       [0.10765   , 0.1104    , 0.07905   , 0.0451    ],
       [0.34883125, 0.07905   , 0.33669375, 0.057825  ],
       [0.049975  , 0.0451    , 0.057825  , 0.0724    ]])}


In [41]:
#Step 2

def LDA(x, mean, avg_cov):
    prob = {}
    prob['Iris-setosa'] = P(x,mean['setosa'],avg_cov)
    prob['Iris-versicolor'] = P(x,mean['versicolor'],avg_cov)
    prob['Iris-virginica'] = P(x,mean['virginica'],avg_cov)
    
    # key is set to prob.get since we need to compare the values of the dictionary, not the keys
    return max(prob, key=prob.get)

In [43]:
# Calculate the accuracy by comparing LDA/QDA prediction vs actual value
def calculate_accuracy(classifier, subset):
    n_correct = 0
    for row in subset.iterrows():
        x = np.array(row[1][0:4])
        actual = row[1][4]
        # classifier = 1: LDA, classifier = 2: QDA, classifier = 3: QDA with independent features
        if classifier == 1:
            if LDA(x, means, cov_avg) == actual:
                n_correct += 1
        elif classifier == 2:
            if QDA(x, means, covs) == actual:
                n_correct += 1
        elif classifier == 3:
            if QDA(x, means, indep_covs) == actual:
                n_correct += 1
        else:
            raise ValueError("Classifier unknown. Please try again")
    accuracy = (n_correct/float(len(subset)) * 100)
    error = 100 - accuracy
    return str(error)
print("Error rate for LDA on training subset: " + calculate_accuracy(1, train) + "%")
print("Error rate for LDA on testing subset: " + calculate_accuracy(1, test) + "%" )

Error rate for LDA on training subset: 2.5%
Error rate for LDA on testing subset: 0.0%


In [44]:
#step3
def QDA(x, mean, covs):
    prob = {}
    prob['Iris-setosa'] = P(x,mean['setosa'],covs['setosa'])
    prob['Iris-versicolor'] = P(x,mean['versicolor'],covs['versicolor'])
    prob['Iris-virginica'] = P(x,mean['virginica'],covs['virginica'])
    
    # key is set to prob.get since we need to compare the values of the dictionary, not the keys
    return max(prob, key=prob.get)

In [45]:
print("Error rate for QDA on training subset: " + calculate_accuracy(2, train) + "%")
print("Error rate for QDA on testing subset: " + calculate_accuracy(2, test) + "%")

Error rate for QDA on training subset: 1.6666666666666714%
Error rate for QDA on testing subset: 0.0%


In [46]:
#step4
# do training & testing sets together, rather than separated
categories = ['Iris-setosa','Iris-versicolor','Iris-virginica']

# run LDA on each separate class
for category in categories:
    flower_class = data[data[4] == category]
    n_correct = 0
    for row in flower_class.iterrows():
        x = np.array(row[1][0:4])
        actual = row[1][4]
        predicted = LDA(x,means,cov_avg)
        # if predicted answer matches our actual answer, we consider that a success
        if predicted == actual:
            n_correct += 1
    accuracy = (n_correct/float(len(flower_class)) * 100)
    error = 100 - accuracy
    print(category, "error rate:", error, "%")

Iris-setosa error rate: 0.0 %
Iris-versicolor error rate: 4.0 %
Iris-virginica error rate: 2.0 %


In [47]:
#step5
#convert cov matrices to diagonal (set non-diag entries to 0)
indep_covs = {}
for category, cov in covs.items():
    
    # setup each category to have a 4 x 4 identity matrix
    indep_covs[category] = np.zeros(cov.shape)
    
    # we should only add the diagonal values from our covariance matrices to our identity matrix
    for row in range(cov.shape[0]):
        for col in range(cov.shape[1]):
            if row == col:
                indep_covs[category][row][col] = cov[row][col]
    print(indep_covs[category])

[[0.12784375 0.         0.         0.        ]
 [0.         0.1294     0.         0.        ]
 [0.         0.         0.02884375 0.        ]
 [0.         0.         0.         0.00969375]]
[[0.2669     0.         0.         0.        ]
 [0.         0.1081     0.         0.        ]
 [0.         0.         0.19844375 0.        ]
 [0.         0.         0.         0.042     ]]
[[0.45624375 0.         0.         0.        ]
 [0.         0.1104     0.         0.        ]
 [0.         0.         0.33669375 0.        ]
 [0.         0.         0.         0.0724    ]]


In [1]:
# calculate the time and error rates for QDA 
qda_start_time = time.time() * 1000
print("Error rate for QDA on training subset: " + calculate_accuracy(2, train) + "%")
print("Error rate for QDA on testing subset: " + calculate_accuracy(2, test) + "%")
print("Time taken for QDA with features:", (time.time()*1000)-qda_start_time, "ms\n")

# calculate the time and error rates for QDA with independent features
indep_start_time = time.time() * 1000
print("Error rate for QDA on training subset with independent features : " + calculate_accuracy(3, train) + "%")
print("Error rate for QDA on testing subset with independent features : " + calculate_accuracy(3, test) + "%")
print("Time taken for QDA with independent features:", (time.time()*1000)-indep_start_time, "ms")


NameError: name 'time' is not defined